# Mobile price classification with multilayer perceptron
## PyTorch
Erik Matovič and Jakub Horvat

[Dataset](https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification?select=train.csv)

### 0. Imports

In [28]:
import wandb
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from utils import split_train_val
from typing import Tuple, List

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### 2. Data transformation
 - no null values
 - load train and test datasets & from test dataset make validation and test dataset

In [29]:
# read datasets
df_train = pd.read_csv('../data/train.csv', sep=',')
df_train

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [30]:
df_test = pd.read_csv('../data/test.csv', sep=',', index_col=0)
df_test

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
id,,,,,,,,,,,,,,,,,,,,
1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0
2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0
3,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1
4,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0
5,1434,0,1.4,0,11,1,49,0.5,108,6,18,749,810,1773,15,8,7,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1700,1,1.9,0,0,1,54,0.5,170,7,17,644,913,2121,14,8,15,1,1,0
997,609,0,1.8,1,0,0,13,0.9,186,4,2,1152,1632,1933,8,1,19,0,1,1
998,1185,0,1.4,0,1,1,8,0.5,80,1,12,477,825,1223,5,0,14,1,0,0


In [31]:
X_train = df_train.values[:, :-1]       # store the inputs
X_train = X_train.astype('float32')     # input data as floats
X_train

array([[8.420e+02, 0.000e+00, 2.200e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [1.021e+03, 1.000e+00, 5.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [5.630e+02, 1.000e+00, 5.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [1.911e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [1.512e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [5.100e+02, 1.000e+00, 2.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00]], dtype=float32)

In [32]:
y_train = df_train.values[:, -1]  # store the outputs
y_train

array([1., 2., 2., ..., 3., 0., 3.])

In [33]:
batch_size = 32
tf_train, tf_val = split_train_val(df_train)

print(tf_train.dataset)
print(tf_val.dataset.shape)

      battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  \
0               842     0          2.2         0   1       0           7   
1              1021     1          0.5         1   0       1          53   
2               563     1          0.5         1   2       1          41   
3               615     1          2.5         0   0       0          10   
4              1821     1          1.2         0  13       1          44   
...             ...   ...          ...       ...  ..     ...         ...   
1995            794     1          0.5         1   0       1           2   
1996           1965     1          2.6         1   0       0          39   
1997           1911     0          0.9         1   1       1          36   
1998           1512     0          0.9         0   4       1          46   
1999            510     1          2.0         1   5       1          45   

      m_dep  mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  \
0       

In [41]:
train_dl = DataLoader(tf_train, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(tf_val, batch_size=batch_size, shuffle=False)
test_dl = DataLoader(df_test, batch_size=batch_size, shuffle=False)

print(len(train_dl.dataset))
print(len(val_dl.dataset))
print(len(test_dl.dataset))

print(train_dl.dataset)
print(val_dl.dataset)
print(test_dl.dataset)

1800
200
1000
      battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  \
id                                                                         
1              1043     1          1.8         1  14       0           5   
2               841     1          0.5         1   4       1          61   
3              1807     1          2.8         0   1       0          27   
4              1546     0          0.5         1  18       1          25   
5              1434     0          1.4         0  11       1          49   
...             ...   ...          ...       ...  ..     ...         ...   
996            1700     1          1.9         0   0       1          54   
997             609     0          1.8         1   0       0          13   
998            1185     0          1.4         0   1       1           8   
999            1533     1          0.5         1   0       0          50   
1000           1270     1          0.5         0   4       1          35  

## Model

In [35]:
class MLP(nn.Module):
    """ 
    Model class.
    :param Module from torch.nn
    """
    def __init__(self, n_inputs: int, n_classes: int, n_epochs: int, lr: float, momentum: float) -> None:
        """
        Model elements init.
        """
        super(MLP, self).__init__()
        self.n_inputs = n_inputs
        self.n_classes = n_classes
        self.n_epochs = n_epochs
        self.lr = lr
        self.momentum = momentum
    
        # input to first hidden layer
        self.dense1 = nn.Linear(n_inputs, 32)
        # second hidden layer
        self.dense2 = nn.Linear(32, 32)
        # third hidden layer and output
        self.dense3 = nn.Linear(32, n_classes)
        # activation f
        self.relu = nn.ReLU()
 
    def forward(self, X):
        """
        Feed forward
        """
        print(type(X))
        # input to first hidden layer
        output = self.dense1(X)
        output = self.relu(X)
        
        # second hidden layer
        output = self.dense2(X)
        output = self.relu(X)
        
        # third hidden layer and output
        output = self.dense(X)

        print(type(output))
        return output

In [36]:
def train_mlp(n_epochs, mlp, optimizer, loss_fn, train_loader, 
              val_loader, device=device) -> Tuple[List, List, List, List]:
    """ 
    Train the model
    """
    # n_epochs = mlp.n_epochs
    # lr = mlp.lr
    # momentum = mlp.momentum
    
    # define the optimization
    # loss_fn = nn.CrossEntropyLoss()
    # optimizer = optim.SGD(mlp.parameters(), lr=lr, momentum=momentum)

    # init train lists for statistics
    loss_train, accuracy_train = list(), list()

    # init validation lists for statistics
    loss_validation, accuracy_validation = list(), list()

    # enumerate epochs
    for epoch in range(n_epochs):
        # init epoch train counters
        epoch_train_accuracy, epoch_train_total, epoch_train_true, epoch_train_loss = 0, 0, 0, 0

        # init epoch validation counters
        epoch_validation_accuracy, epoch_validation_total, \
            epoch_validation_true, epoch_validation_loss = 0, 0, 0, 0

        # enumerate mini batches
        for idx, (X_batch, y_batch) in enumerate(train_loader):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            y_hat = mlp(X_batch)
            # calculate loss
            loss = loss_fn(y_hat, y_batch)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

            # update train counters
            epoch_train_loss += loss.item()
            epoch_train_true += (y_batch == y_hat).sum().item()
            epoch_train_total += len(y_batch)
        
        # update train accuracy & loss statistics
        epoch_train_accuracy = epoch_train_true/epoch_train_total
        epoch_train_loss /= (len(train_loader.dataset)/batch_size)

        # disable gradient calculation
        with torch.no_grad():
            # enumerate mini batches
            for idx, (X_batch, y_batch) in enumerate(val_loader):
                # compute the models output
                y_hat = mlp(X_batch)
                # calculate loss
                loss = loss_fn(y_hat, y_batch)

                # update validation counters
                epoch_validation_loss += loss.item()
                epoch_validation_true += (y_batch == y_hat).sum().item()
                epoch_validation_total += len(y_batch)
        
        # update validation accuracy & loss statistics
        epoch_validation_accuracy = epoch_validation_true/epoch_validation_total
        epoch_validation_loss /= (len(val_loader.dataset)/batch_size)

        # update global epochs statistics
        loss_train.append(epoch_train_loss)
        accuracy_train.append(epoch_train_accuracy)
        loss_validation.append(epoch_validation_loss)
        accuracy_validation.append(epoch_validation_accuracy)

        # WandB log
        # TO DO

        # print
        print(
            f'Epoch {epoch}/{n_epochs}: \
            \t train loss {loss_train[-1]}, \
            \t validation loss {loss_validation[-1]}, \
            \t train accuracy {accuracy_train[-1]}, \
            \t validation accuracy {accuracy_validation[-1]}'
        )

    return loss_train, accuracy_train, loss_validation, accuracy_validation

In [37]:
import warnings
warnings.filterwarnings('ignore')

n_epochs = 10 
learning_rate = 0.001 
momentum = 0.9

input_size = 20          # number of features
hidden_size = 32        # number of features in hidden state

num_classes = 4 # podla sequencies co mame 8 # 50 # number of output classes 

mlp = MLP( 
    n_inputs=input_size, 
    n_classes=num_classes, 
    n_epochs=n_epochs,
    lr=learning_rate,
    momentum=momentum) #.cuda()
# mlp.to(device)

In [38]:
mlp

MLP(
  (dense1): Linear(in_features=20, out_features=32, bias=True)
  (dense2): Linear(in_features=32, out_features=32, bias=True)
  (dense3): Linear(in_features=32, out_features=4, bias=True)
  (relu): ReLU()
)

In [39]:
# define the optimization
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=learning_rate, momentum=momentum)

In [40]:
loss_train, accuracy_train, loss_validation, accuracy_validation = train_mlp(
    n_epochs=n_epochs,
    mlp=mlp,
    optimizer=optimizer,
    loss_fn=loss_fn,
    train_loader=train_dl,
    val_loader=val_dl,
    device=device
)

KeyError: 408